In [66]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from statsmodels.stats.multitest import multipletests

In [67]:
import requests
from xml.etree import ElementTree as ET

def fetch_gene_info(chromosome, start, end):
    query_xml = f"""
    <Query virtualSchemaName="default" formatter="TSV" header="0" uniqueRows="1" count="" datasetConfigVersion="0.6">
        <Dataset name="hsapiens_gene_ensembl" interface="default">
            <Filter name="chromosome_name" value="{chromosome}"/>
            <Filter name="start" value="{start}"/>
            <Filter name="end" value="{end}"/>
            <Attribute name="ensembl_gene_id"/>
            <Attribute name="external_gene_name"/>
            <Attribute name="description"/>
            <Attribute name="start_position"/>
            <Attribute name="end_position"/>            
            <Attribute name="strand"/>
        </Dataset>
    </Query>
    """

    biomart_url = "http://www.ensembl.org/biomart/martservice?query="
    response = requests.get(biomart_url + query_xml.strip())

    if response.status_code != 200:
        raise Exception(f"Error fetching data from BioMart: {response.text}")

    genes = [line.split("\t") for line in response.text.strip().split("\n")]
    print(genes)
    return genes


In [68]:
split_race_region={'ACB': {},
 'ASW': {},
 'BEB': {'split_chr3/xav': [[98046342, 98273693]]},
 'CDX': {'split_chr13/xai': [[55102733, 55443491]]},
 'CEU': {'split_chr17/xaj': [[45670177, 46418024]]},
 'CHB': {},
 'CHS': {'split_chr3/xav': [[98046342, 98273693]]},
 'CLM': {'split_chr17/xaj': [[45836662, 46224960]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ESN': {'split_chr5/xai': [[34264948, 34562837]]},
 'FIN': {},
 'GBR': {'split_chr1/xbt': [[226978066, 227352188]],
  'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'GIH': {'split_chr3/xav': [[98046342, 98276757]]},
 'GWD': {},
 'IBS': {'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ITU': {'split_chr17/xak': [[52575391, 52889013]]},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {'split_chr5/xai': [[34264948, 34562837]]},
 'MXL': {'split_chr1/xbt': [[227016487, 227352188]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'PEL': {'split_chr11/xaf': [[23168664, 23564494]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'PJL': {},
 'PUR': {'split_chr17/xaj': [[45836662, 46418024]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'STU': {'split_chr13/xai': [[55102733, 55443491]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'TSI': {'split_chr17/xaj': [[45670177, 46418024]]},
 'YRI': {}}

In [69]:
def generate_gene_info(gene_list_one_record,chrnum):
    result=[]
    result.append(gene_list_one_record[0])
    result.append(gene_list_one_record[1])
    s=gene_list_one_record[3]
    e=gene_list_one_record[4]
    
    pos="chr"+chrnum+":"+str(s)+"-"+str(e)
    result.append(pos)
    result.append(gene_list_one_record[5])
    result.append(gene_list_one_record[2])
    
    return result

In [70]:
import re

def extract_numbers(input_string):
    return re.findall(r'\d+', input_string)

In [71]:
def ifgeneinside_region(complementary_region,gene_region):
    
    s=int(gene_region[0])
    e=int(gene_region[1])
     
    if (complementary_region[0]<=s) and complementary_region[1]>=e:
        return True
    else:
        return False

In [72]:

allrecords=[]

#non_mhc
for race,split_ in split_race_region.items():
    if race!="All":
        for chr,arr in split_.items():
            chr=chr.split("/")[0].split("_")[1]
            chr=extract_numbers(chr)[0]
            print(chr)
            for a in arr:
                gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                for gene_list_record in gene_list:
                    if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                        onerecord=[race,0]
                        onerecord+=["chr"+chr+":"+str(a[0])+"-"+str(a[1])]
                        
                        onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum=chr)
                        allrecords.append(onerecord)
                    else:
                        continue
                


3


[['ENSG00000250750', 'OR5BM1P', 'olfactory receptor family 5 subfamily BM member 1 pseudogene [Source:HGNC Symbol;Acc:HGNC:14835]', '98053374', '98053739', '1'], ['ENSG00000213439', 'OR5AC1', 'olfactory receptor family 5 subfamily AC member 1 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:15047]', '98064472', '98065396', '1'], ['ENSG00000196578', 'OR5AC2', 'olfactory receptor family 5 subfamily AC member 2 [Source:HGNC Symbol;Acc:HGNC:15431]', '98087173', '98088102', '1'], ['ENSG00000249225', '', 'novel transcript, antisense to OR5H1, OR5H14, OR5H15', '98099908', '98148134', '-1'], ['ENSG00000290819', '', 'novel transcript', '98100793', '98105672', '1'], ['ENSG00000251090', 'OR5AC4P', 'olfactory receptor family 5 subfamily AC member 4 pseudogene [Source:HGNC Symbol;Acc:HGNC:31284]', '98104754', '98105672', '1'], ['ENSG00000248365', 'POU5F1P7', 'POU class 5 homeobox 1 pseudogene 7 [Source:HGNC Symbol;Acc:HGNC:33313]', '98119191', '98119731', '1'], ['ENSG00000231192', 'OR5H1', 'olfactory

In [73]:
mhc_race_positions_dict={'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'All':{'mhc':[[29720403, 30048796],[30994370, 31528792],[32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30048796],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[30994370, 31528792], [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'GWD': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'IBS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ITU': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'JPT': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'KHV': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32397207, 32923168]]},
 'LWK': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32212726, 32923168]]},
 'MSL': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'MXL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'PEL': {'mhc': [[29720403, 29896285],
   [30959575, 31528792],
   [32288923, 32882258]]},
 'PJL': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'PUR': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'STU': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'TSI': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'YRI': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]}}

In [74]:
allrecords

[['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000250750',
  'OR5BM1P',
  'chr3:98053374-98053739',
  '1',
  'olfactory receptor family 5 subfamily BM member 1 pseudogene [Source:HGNC Symbol;Acc:HGNC:14835]'],
 ['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000213439',
  'OR5AC1',
  'chr3:98064472-98065396',
  '1',
  'olfactory receptor family 5 subfamily AC member 1 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:15047]'],
 ['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000196578',
  'OR5AC2',
  'chr3:98087173-98088102',
  '1',
  'olfactory receptor family 5 subfamily AC member 2 [Source:HGNC Symbol;Acc:HGNC:15431]'],
 ['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000249225',
  '',
  'chr3:98099908-98148134',
  '-1',
  'novel transcript, antisense to OR5H1, OR5H14, OR5H15'],
 ['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000290819',
  '',
  'chr3:98100793-98105672',
  '1',
  'novel transcript'],
 ['BEB',
  0,
  'chr3:98046342-98273693',
  'ENSG00000251090',
  'OR5AC4P

In [75]:

for race, split_ in mhc_race_positions_dict.items():
    if split_!={}:
        if race!="All":
            for chr,arr in split_.items():
                chr="6" 
                     
                for a in arr:
                    gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                    for gene_list_record in gene_list:
                        if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                            onerecord=[race,1]
                            onerecord+=[chr+":"+str(a[0])+"-"+str(a[1])]
                            
                            onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum="6")
                            allrecords.append(onerecord)   
                        else:
                            continue                
        else:
            for chr,arr in split_.items():
                chr="6"
                for a in arr:
                    gene_list=fetch_gene_info(chromosome=chr,start=a[0]-1,end=a[1]-1)
                    for gene_list_record in gene_list:
                        if ifgeneinside_region(complementary_region=a,gene_region=[gene_list_record[3],gene_list_record[4]]):
                            onerecord=[race,1]
                            onerecord+=[chr+":"+str(a[0])+"-"+str(a[1])]
                            
                            onerecord+=generate_gene_info(gene_list_one_record=gene_list_record,chrnum="6")
                            allrecords.append(onerecord)    
                        else:
                            continue

[['ENSG00000204642', 'HLA-F', 'major histocompatibility complex, class I, F [Source:HGNC Symbol;Acc:HGNC:4963]', '29722775', '29738528', '1'], ['ENSG00000225864', '', 'HLA complex group 4 pseudogene 11', '29722981', '29723971', '-1'], ['ENSG00000214922', 'HLA-F-AS1', 'HLA-F antisense RNA 1 [Source:HGNC Symbol;Acc:HGNC:26645]', '29726601', '29749049', '-1'], ['ENSG00000239257', 'RPL23AP1', 'ribosomal protein L23a pseudogene 1 [Source:HGNC Symbol;Acc:HGNC:10318]', '29726669', '29727139', '-1'], ['ENSG00000273340', 'MICE', 'MHC class I polypeptide-related sequence E (pseudogene) [Source:HGNC Symbol;Acc:HGNC:7094]', '29741731', '29748969', '-1'], ['ENSG00000227758', 'HCG9P5', 'HLA complex group 9 pseudogene 5 [Source:HGNC Symbol;Acc:HGNC:30983]', '29748289', '29748513', '1'], ['ENSG00000199290', 'Y_RNA', 'Y RNA [Source:RFAM;Acc:RF00019]', '29750371', '29750472', '1'], ['ENSG00000235821', 'IFITM4P', 'interferon induced transmembrane protein 4 pseudogene [Source:HGNC Symbol;Acc:HGNC:21669]',

In [76]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    #dataframe(dictforDF).to_csv("csv2_05_09.csv")
    return dataframe(dictforDF)


In [77]:
dfnew=generatedf(columns=["Race","isMHC","Position of disassortative mating region","Gene ID","Gene name","Position of gene","is_complement","Gene description"],allrecords=allrecords)

In [78]:
dfnew['is_complement'] = dfnew['is_complement'].apply(lambda x:  "F" if x == "1" else "R")
# dfnew.to_csv("csv2_05_11new.csv")

In [79]:
dfnew

,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description
0,BEB,0,chr3:98046342-98273693,ENSG00000250750,OR5BM1P,chr3:98053374-98053739,F,olfactory receptor family 5 subfamily BM membe...
1,BEB,0,chr3:98046342-98273693,ENSG00000213439,OR5AC1,chr3:98064472-98065396,F,olfactory receptor family 5 subfamily AC membe...
2,BEB,0,chr3:98046342-98273693,ENSG00000196578,OR5AC2,chr3:98087173-98088102,F,olfactory receptor family 5 subfamily AC membe...
3,BEB,0,chr3:98046342-98273693,ENSG00000249225,,chr3:98099908-98148134,R,"novel transcript, antisense to OR5H1, OR5H14, ..."
4,BEB,0,chr3:98046342-98273693,ENSG00000290819,,chr3:98100793-98105672,F,novel transcript
...,...,...,...,...,...,...,...,...
3116,YRI,1,6:32212726-32882258,ENSG00000204264,PSMB8,chr6:32840717-32844679,R,proteasome 20S subunit beta 8 [Source:HGNC Sym...
3117,YRI,1,6:32212726-32882258,ENSG00000204261,PSMB8-AS1,chr6:32844078-32846500,F,PSMB8 antisense RNA 1 (head to head) [Source:H...
3118,YRI,1,6:32212726-32882258,ENSG00000240065,PSMB9,chr6:32844136-32859851,F,proteasome 20S subunit beta 9 [Source:HGNC Sym...
3119,YRI,1,6:32212726-32882258,ENSG00000168394,TAP1,chr6:32845209-32853816,R,"transporter 1, ATP binding cassette subfamily ..."


In [80]:
dfnew.to_csv("csv2_first_with_repeat_from_ipynb.csv")
